# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['HeadBack', 'Left', 'Crouch', 'StandInit', 'Knee', 'Frog', 'Back', 'Stand', 'Sit', 'Belly', 'Right']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(0,len(classes)):
    data, target= load_pose_data(i)
    all_data+=data
    all_target+= target
print 'total number of data', len(all_data)

total number of data 223


In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [7]:
# YOUR CODE HERE
import copy
permuted_training_data=[]
permuted_training_target=[]
test_data=[]
test_target=[]
count_deleted=copy.deepcopy(all_target)
for i in training_data:
    permuted_training_data.append(all_data[i])
    permuted_training_target.append(all_target[i])
    count_deleted[i]=-1
clf.fit(permuted_training_data,permuted_training_target)

for i in range(0,len(count_deleted)):
    if count_deleted[i]==-1:
        continue
    test_data.append(all_data[i])
    test_target.append(all_target[i])


### predicting

In [13]:
clf.predict(all_data[-1:]), all_target[-1]

(array([2]), 10)

In [14]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [15]:

expected = []
predicted = []

predicted= clf.predict(permuted_training_data)
expected=permuted_training_target

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         7
          1       1.00      1.00      1.00        14
          2       1.00      1.00      1.00        24
          3       1.00      1.00      1.00        38
          4       1.00      1.00      1.00         6
          5       1.00      1.00      1.00         7
          6       1.00      1.00      1.00        16
          7       1.00      1.00      1.00         9
          8       1.00      1.00      1.00        13
          9       1.00      1.00      1.00        15
         10       1.00      1.00      1.00         7

avg / total       1.00      1.00      1.00       156


Confusion matrix:
[[ 7  0  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0  0]
 [ 0  0 24  0  0  0  0  0  0  0  0]
 [ 0  0  0 38  0  0  0  0  0  0  0]
 [ 0  0  0  0  6  0  0  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0]
 [ 0  0  0  0 

## 4. Evaluate on the test data

In [16]:
expected = []
predicted = []
predicted= clf.predict(test_data)
expected=test_target

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         3
          1       1.00      1.00      1.00         6
          2       0.86      1.00      0.92         6
          3       1.00      1.00      1.00        14
          4       1.00      1.00      1.00         4
          5       1.00      1.00      1.00         3
          6       1.00      1.00      1.00         8
          7       1.00      1.00      1.00         2
          8       1.00      1.00      1.00        13
          9       1.00      1.00      1.00         4
         10       1.00      0.75      0.86         4

avg / total       0.99      0.99      0.98        67


Confusion matrix:
[[ 3  0  0  0  0  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0  0  0  0  0]
 [ 0  0  6  0  0  0  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0  0  0  0]
 [ 0  0  0  0  4  0  0  0  0  0  0]
 [ 0  0  0  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  8  0  0  0  0]
 [ 0  0  0  0 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [17]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [19]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([2]), [10])